# 5. Reranking de documentos en sistemas RAG

La recuperación inicial se basa en similitud entre embeddings: la relevancia se calcula de forma **independiente** (query y cada documento por separado). El **reranking** añade una segunda etapa: toma los candidatos del retriever y los reordena con un modelo que evalúa **pares (consulta, documento)** de forma conjunta, para quedarse solo con los más relevantes.

Se aplica **después** de la recuperación y **antes** de la generación de la respuesta. Los **cross-encoders** son adecuados aquí porque solo se aplican a un número reducido de documentos (p. ej. 10), donde la precisión importa más que la escalabilidad.

⚠️ **Requisito**: Ejecute antes el notebook `03_rag_base.ipynb` para generar la carpeta `faiss_index`. Este notebook carga ese índice y crea un retriever con `k=10` para la etapa de reranking.

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from sentence_transformers import CrossEncoder
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

embeddings_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Cargar índice FAISS creado en el notebook 03 y crear retriever con k=10 (más candidatos para reranking)
vectorstore = FAISS.load_local("./faiss_index", embeddings_model, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})
print("Retriever configurado con k=10.")

## Creación del reranker local

Se utiliza un **cross-encoder** de `sentence-transformers` (modelo liviano entrenado sobre MS MARCO) para evaluar pares (query, documento) y asignar un puntaje de relevancia.

In [ ]:
def create_local_reranker(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2"):
    """
    Crea y devuelve un modelo CrossEncoder para realizar reranking local
    de documentos previamente recuperados.

    Args:
        model_name (str): Nombre o ruta del modelo cross-encoder. Por defecto
            un modelo liviano optimizado para búsqueda semántica.

    Returns:
        CrossEncoder: Instancia del modelo lista para el proceso de reranking.
    """
    cross_encoder = CrossEncoder(model_name)
    print(f"Reranker: {model_name}")
    return cross_encoder

cross_encoder = create_local_reranker()

## Función de reranking

Reordena los documentos recuperados según el puntaje del cross-encoder y devuelve los `n` más relevantes (`n` debe ser menor que el `k` usado en el retriever).

In [ ]:
def rerank_documents(reranker, question, retrieved_docs, n=3, debug=False):
    """
    Reordena los documentos recuperados usando un cross-encoder y devuelve
    los n más relevantes respecto a la pregunta.

    Args:
        reranker: Modelo cross-encoder para relevancia (query, documento).
        question (str): Pregunta del usuario.
        retrieved_docs (list): Documentos devueltos por el retriever.
        n (int): Número de documentos finales a retornar.
        debug (bool): Si True, imprime puntajes por documento.

    Returns:
        list: Los n documentos más relevantes, ordenados de mayor a menor puntaje.
    """
    pairs = [(question, doc.page_content) for doc in retrieved_docs]
    scores = reranker.predict(pairs)
    scored_docs = list(zip(retrieved_docs, scores))
    reranked = sorted(scored_docs, key=lambda x: x[1], reverse=True)

    if debug:
        print("\nPUNTAJES ASIGNADOS POR EL RERANKER:")
        print("=" * 70)
        for i, (doc, score) in enumerate(reranked, 1):
            file = doc.metadata.get("source_file", "unknown")
            preview = doc.page_content[:60].replace("\n", " ")
            print(f"\n{i}. [{file}] '{preview}...'")
            print(f"   Relevance score: {score:.4f}")
        print("\n" + "-" * 40)

    return [doc for doc, _ in reranked[:n]]

## Demostración del proceso de reranking

Se ejecuta una consulta: el retriever devuelve 10 candidatos y el reranker los reordena y filtra a los más relevantes.

In [ ]:
def demo_reranker(retriever, reranker, query):
    """
    Ejecuta búsqueda con el retriever, aplica reranking y muestra los documentos
    finales ordenados por relevancia.

    Args:
        retriever: Retriever configurado (p. ej. FAISS con k=10).
        reranker: Modelo cross-encoder para reranking.
        query (str): Consulta de ejemplo.
    """
    docs = retriever.invoke(query)
    reranked_docs = rerank_documents(reranker, query, docs, n=3, debug=True)

    print("\nRESULTADOS RETRIEVER + RERANKER:")
    print("=" * 70)
    print(f"\nConsulta: '{query}'")
    print(f"Documentos finales: {len(reranked_docs)}")
    for i, doc in enumerate(reranked_docs, 1):
        file = doc.metadata.get("source_file", "unknown")
        preview = doc.page_content[:60].replace("\n", " ")
        print(f"\n  {i}. {file}: '{preview}...'")
    print("\n" + "-" * 30)

In [ ]:
demo_reranker(retriever, cross_encoder, "¿Qué prácticas ayudan a mejorar la robustez del manejo de errores y autenticación en APIs?")